In [1]:
pip install opencv-contrib-python==4.5.3.56

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import cv2 as cv # computer vision library for image processing
import pyttsx3 # for text to speech conversion
import numpy as np

In [3]:
# Distance constants 
KNOWN_DISTANCE = 114.3 #CENTIMETERS
PERSON_WIDTH = 37 #CENTIMETERS
MOBILE_WIDTH = 7 #CENTIMETERS
BOTTLE_WIDTH = 8 #CENTIMETERS

# Object detector constant 
CONFIDENCE_THRESHOLD = 0.5
NMS_THRESHOLD = 0.3

# colors for object detected
COLORS = [(255,0,0),(255,0,255),(0, 255, 255), (255, 255, 0), (0, 255, 0), (255, 0, 0)]
# COLORS = [red, purple, cyan, brown, green, red] 

GREEN =(0,255,0) # To show the distance text above the object in GREEN
BLACK =(0,0,0) # To show the bounding box (rectangle) in BLACK

# defining fonts 
FONTS = cv.FONT_HERSHEY_COMPLEX

# getting class names from classes.txt file 
class_names = []
#with open("C:\\Users\\KIIT\\MyPython\\PROJECTS\\object_detection+dist\\Yolov4-Detector-and-Distance-Estimator-master\\classes.txt", "r") as f:
with open("classes.txt", "r") as f:
    class_names = [cname.strip() for cname in f.readlines()]

# setting up opencv net

yoloNet = cv.dnn.readNet('yolov4-tiny.weights', 'yolov4-tiny.cfg')
# Reads deep learning network represented in one of the supported formats.
# For more info, check the links below:
# https://docs.opencv.org/3.4/d6/d0f/group__dnn.html#ga3b34fe7a29494a6a4295c169a7d32422
# https://stackoverflow.com/questions/50390836/understanding-darknets-yolo-cfg-config-files

yoloNet.setPreferableBackend(cv.dnn.DNN_BACKEND_CUDA)
yoloNet.setPreferableTarget(cv.dnn.DNN_TARGET_CUDA_FP16)

model = cv.dnn_DetectionModel(yoloNet)

'''
scale -> scale factor (1/255 to scale the pixel values to [0..1])
size -> 416x416 square image

(swapBR=True) -> since OpenCV uses BGR. 
    OpenCV assumes images are in BGR channel order; 
    however, the `mean` value assumes we are using RGB order. 
    To resolve this discrepancy we can swap the R and B channels in image
    by setting this value to `True`.
    By default OpenCV performs this channel swapping for us.
'''
model.setInputParams(size=(416, 416), scale=1/255, swapRB=True)

In [4]:
# object detector function /method
def object_detector(image):
    classes, scores, boxes = model.detect(image, CONFIDENCE_THRESHOLD, NMS_THRESHOLD)
    # creating empty list to add objects data
    data_list = []
    for (classid, score, box) in zip(classes, scores, boxes):
        
        # define color of each, object based on its class id
        '''
        80 object names are stored in classes.txt and 6 colours are stored in var COLORS
        For 1st object name (person) -> 0 % 6 = 0. So colour[0] = RED will be chosen
        For 2nd object name (bicycle) -> 1 % 6 = 1. So colour[1] = PURPLE will be chosen
        And so on...
        '''
        color= COLORS[int(classid) % len(COLORS)]
        
        # label -> stores class id and confidence score
        label = "%s : %f" % (class_names[classid[0]], score)

        # draw rectangle on and label on object
        cv.rectangle(image, box, color, 2)
        cv.putText(image, label, (box[0], box[1]-14), FONTS, 0.5, color, 2)
    
        # getting the data 
        # 1: class name  2: object width in pixels, 3: position where have to draw text(distance)
        if classid == 0: # person class id
            data_list.append([class_names[classid[0]], box[2], (box[0], box[1]-2)])
        elif classid == 39: # bottle class id
            data_list.append([class_names[classid[0]], box[2], (box[0], box[1]-2)])
        elif classid == 67: # cell phone class id
            data_list.append([class_names[classid[0]], box[2], (box[0], box[1]-2)])
        # if you want include more classes then you have to simply add more [elif] statements here
        # returning list containing the object data. 
    return data_list

In [5]:
# FOR FINDING FOCAL LENGTH OF CAM LENS
def focal_length_finder (measured_distance, real_width, width_in_rf):
    focal_length = (width_in_rf * measured_distance) / real_width
    print("FOCAL LENGTH: ", focal_length)
    # focal_length = 560
    return focal_length

In [6]:
# FOR FINDING DISTANCE OF OBJECT FROM CAM
def distance_finder(focal_length, real_object_width, width_in_frame):
    distance = (real_object_width * focal_length) / width_in_frame
    return distance

In [7]:
# FOR FINDING FOCAL LENGTH OF THE CAMERA LENS

# reading the reference image from dir 
ref_person = cv.imread('ReferenceImages/image14.png')
ref_mobile = cv.imread('ReferenceImages/image4.png')

mobile_data = object_detector(ref_mobile)
'''
displays [['person', 440, (50, 134)], ['cell phone', 72, (206, 296)]]
440 -> pixel width of person ; (50, 134) -> coordinates of person in image
72 -> pixel width of person ; (206, 296) -> coordinates of cell phone in image
'''
print(mobile_data)
'''
[['person', 440, (50, 134)], ['cell phone', 72, (206, 296)]]
If we check indices [1][1] => 72
So mobile_data[1][1] = 72
'''
mobile_width_in_rf = mobile_data[1][1]
person_data = object_detector(ref_person)

'''
displays [['person', 367, (135, 134)]]
'''
print(person_data)
person_width_in_rf = person_data[0][1]

print(f"Person width in pixels : {person_width_in_rf}) mobile width in pixel: {mobile_width_in_rf}")
# finding focal length 
focal_length = focal_length_finder(KNOWN_DISTANCE, PERSON_WIDTH, person_width_in_rf)
#focal_length_mobile = focal_length_finder(KNOWN_DISTANCE, MOBILE_WIDTH, mobile_width_in_rf)

[['person', 440, (50, 134)], ['cell phone', 72, (206, 296)]]
[['person', 367, (135, 134)]]
Person width in pixels : 367) mobile width in pixel: 72
FOCAL LENGTH:  1133.7324324324325


In [8]:
def text_to_speech(string):
    text_speech = pyttsx3.init()
    text_speech.say(string)
    text_speech.runAndWait()

In [9]:
def calc_announce_store_dist(obj_info, width):
    print(obj_info)
    dist = distance_finder(focal_length, width, obj_info[1])
    approx_dist = round(dist, 0)
    dist_text = f'{obj_info[0]} detected {approx_dist} cm'
    text_to_speech(dist_text)
    return dist # required for drawing bounding box later

In [10]:
# ACTIVATING AND IMPLEMENTING IN CAMERA

cap = cv.VideoCapture(0) # capturing using cam of pc

while True:
    ret, frame = cap.read()
    
    '''
    if a person comes in front of the camera, it displays 
    [['person', 532, (54, 124)]]
    Same for other objects
    '''
    data = object_detector(frame) 
    #print(data)
    
#     for d in data:
        
#         if d[0] == 'person':
#             x, y = d[2]
#             distance = calc_announce_store_dist(d, PERSON_WIDTH)
        
#         elif d[0] == 'cell phone':
#             x, y = d[2]
#             distance = calc_announce_store_dist(d, MOBILE_WIDTH)
            
#         elif d[0] == 'bottle':
#             x, y = d[2]
#             distance = calc_announce_store_dist(d, BOTTLE_WIDTH)
            
#         cv.rectangle(frame, (x, y-3), (x+150, y+23), BLACK, -1)
#         cv.putText(frame, f'DIST: {round(distance, 2)} CM', (x+5,y+13), FONTS, 0.48, GREEN, 2)    
    
    cv.imshow('frame', frame)
    
    key = cv.waitKey(1)
    if key == ord('q'):
        break
cv.destroyAllWindows()
cap.release()